<a href="https://colab.research.google.com/github/amyth18/CS598-Deep-Learning-Final-Project/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project_kbr_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
! pip install gensim --upgrade

In [36]:
!pip install transformers

In [37]:
import pandas as pd
import torch

In [38]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [39]:
# read data
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

Negative filters (e.g. "no sign of breath problem").

Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

Generate TF-IDF vector for each symptom entity.

Generate multi-hot encoding for y


## Utility Routines For Data Processing

In [40]:
from typing import List
import re
import nltk
from transformers import AutoTokenizer, pipeline,  AutoModelForTokenClassification

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
        tokenizer = AutoTokenizer.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        model = AutoModelForTokenClassification.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        symptom_extractor = pipeline('ner', model=model, tokenizer=tokenizer,
                                     device=0)
        text_anno = symptom_extractor(self.dframe["TEXT"].tolist(), 
                                           batch_size=256)
        self.data = list(zip(text_anno, self.dframe["TEXT"].tolist()))
        


    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    def extract_symptoms(self, input_anno_text) -> List[str]:
      """
      The method extract the symptom phrases from the input
      """
      extractions, text = input_anno_text
      span_extract = []
      for extract in extractions:
        if 'problem' in extract['entity']:
          span_extract.append((extract['start'], extract['end']))

      # Check if this span_extract is empty
      if not span_extract:
        return []
      
      span_st = span_extract[0][0]
      final_span = []
      final_end = span_extract[0][1]
      for idx, (st, end) in enumerate(span_extract):
        if idx == 0:
          continue
        if st - span_extract[idx-1][1] <= 2:
          final_end = end
          if idx == len(span_extract) - 1:
            final_span.append((span_st, final_end))
        else:
          final_span.append((span_st, final_end))
          span_st = st
          final_end = end

      text_extracts = [text[st:end].replace("\n", " ") for st, end in final_span]
      return text_extracts

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          symptoms = " ".join(self.extract_symptoms(self.data[idx]))
          text = self.sanitize_text(symptoms)
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
def pad_dataset(dataset):
  seq_lengths = list()

  for idx in range(len(dataset)):
    seq_lengths.append(len(dataset[idx]))
  max_seq_length = max(seq_lengths)
  data_point_len = max(len(elem[0]) for elem in dataset if elem)
  padded_dataset = torch.zeros([len(dataset), max_seq_length, data_point_len], 
                              dtype=torch.float)
  for i in range(len(dataset)):
    for j in range(len(dataset[i])):
      padded_dataset[i][j] = torch.FloatTensor(dataset[i][j])
  
  return padded_dataset

## Data Filtering and Tranformation

In [42]:
df_icd_codes = pd.read_csv(
    "/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

Get top #50 ICD9 codes 

In [43]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()

Filter data to include admimission with top 50 diseases only and group and reorganize data in the following format <subject_id, hadm_id, [icd_code1, icd_code2 ...]>

In [44]:
df_admissions_with_top_diseases = \
df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]

df_admissions_with_top_diseases = \
df_admissions_with_top_diseases.groupby(
['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].apply(
        list).to_frame().reset_index()

Now select discharge summaries for the admimissions that contain atleast one of the top #50 ICD codes.

In [45]:
df_dataset = pd.merge(df, df_admissions_with_top_diseases, 
                       on=["SUBJECT_ID", "HADM_ID"])

df_dataset = df_dataset[df_dataset["CATEGORY"] 
                          == 'Discharge summary'].reset_index()
# free up some memory
# del df

In [46]:
len(df_dataset)

55988

Need to covert the ICD9 column to multi-hot encoding, we keep the old column with list of codes and added and new column with multi-hot encoding representation.

In [47]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

In [48]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_dataset)):
  icd_codes = df_dataset.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

# new add a new column with multi-hot encoding.
df_dataset['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

Extract symptoms from the text. Note: currently we treat all tokens as symptoms need to add all the filters discussed in the paper later. So we added a column called "SYMPTOMS" which is simply tokenized form of "TEXT" after basic sanitization.

In [49]:
from tqdm import tqdm
sgen = MySentences(df_dataset)
symptom_col = list()
for s in tqdm(sgen):
  symptom_col.append(s)

# add the new column to the dataset.
df_dataset["SYMPTOMS"] = symptom_col

55988it [00:16, 3332.82it/s]


In [50]:
df_dataset = df_dataset[df_dataset.apply(lambda x: len(x.SYMPTOMS) > 0, axis=1)]

In [58]:
df_dataset = df_dataset.reset_index()

In [60]:
df_dataset.to_csv("/content/drive/My Drive/DLH Final Project/kbr_df_dataset.csv")

In [72]:
df_dataset = pd.read_csv("/content/drive/My Drive/DLH Final Project/kbr_df_dataset.csv")

In [98]:
del sgen

## Generate Word2Vec Embeddings

Word2Vec training using gensim.

In [51]:
# NOTE: commenting this part so that we dont run this by mistake.

# import gensim
# sgen = MySentences(df_dataset) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model")

## Construct dataset with Word2Vec embeddings

In [61]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model')

In [62]:
from tqdm import tqdm
X_word2vec = list()
for idx in tqdm(range(len(df_dataset))):
  # ignore words in not vocabulary
  symptoms = df_dataset["SYMPTOMS"][idx]
  symptoms_emb = [model.wv[s] for s in symptoms if s in model.wv]
  X_word2vec.append(symptoms_emb)

# pad the dataset.
# X_word2vec = pad_dataset(X_word2vec)

100%|██████████| 54939/54939 [00:06<00:00, 8084.53it/s]


In [20]:
# import pickle
# pfile = open("/content/drive/My Drive/DLH Final Project/X_word2vec_kbr", "ab")
# pickle.dump(X_word2vec, pfile)
# pfile.close()

# Construct data with TF-IDF encoding

In [63]:
import numpy as np
import itertools

vocab_size = len(model.wv)
tf = np.zeros((len(model.wv), len(top_icd_codes)))


for idx in range(len(df_dataset)):
  # XXX: TODO currently we treat all tokens from "TEXT" as sypmtoms
  # get the icd codes for this discharge summary
  symptoms = df_dataset['SYMPTOMS'][idx]
  icd_codes = df_dataset['ICD9_CODE'][idx]
  # create a cross product of symptoms and icd codes
  # and update tf matrix. tf matrix keeps count of how many 
  # (i.e frequency) times <symptom, icd code> pair occur in our dataset.
  for pair in itertools.product(symptoms, icd_codes):
    # update count of each (symptom, icd_code) pair to compute TF
    if pair[0] in model.wv:
      tf[model.wv.get_index(pair[0])][icd_code_to_idx[pair[1]]] += 1

# Complete the TF-IDF matrix computation.
# Compute the number of ICD Codes (i.e diseaes) each 
# symptom is associated with.
D_i = np.sum(tf > 0, axis=1)
print(D_i.shape)

log_N_Di = np.log(len(top_icd_codes)/D_i)
tf_idf = (tf.T * log_N_Di).T
print(tf_idf.shape)

(64259,)
(64259, 50)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


In [64]:
# build the X_tfidf dataset
X_tf_idf = list()
for idx in range(len(df_dataset)):
  symptoms = df_dataset["SYMPTOMS"][idx]
  # get tf-idf vector for each symptom
  # ignore words in not vocabulary
  symptoms_tf_idf = [tf_idf[model.wv.get_index(s)] \
                     for s in symptoms if s in model.wv]
  X_tf_idf.append(symptoms_tf_idf)

# pad the dataset.
# X_tf_idf = pad_dataset(X_tf_idf)

In [ ]:
# import pickle
# pfile = open("/content/drive/My Drive/DLH Final Project/X_tf_idf", "ab")
# pickle.dump(tf_idf, pfile)
# pfile.close()

# Construct Y (Multihot Encoding)

In [65]:
# multi-hot encoding for ICD codes diagnosed.
y = df_dataset['ICD9_CODE_ENCODED'].to_list()

In [67]:
print(len(X_word2vec))
print(len((X_tf_idf)))
print(len(y))

54939
54939
54939


# Define Dataset and DataLoaders

In [99]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

def collate_fn(data):
  x_w2v, x_tidf, y_batch = zip(*data)
  x_w2v = pad_dataset(x_w2v).cuda()
  x_tidf = pad_dataset(x_tidf).cuda()
  y_batch = torch.FloatTensor(y_batch).cuda()
  # print(f"{x_w2v.shape}, {x_tidf.shape}, {y_batch.shape}")
  return x_w2v, x_tidf, y_batch


class CustomDataset(Dataset):

  def __init__(self, X_w2v, X_tfidf, y):              
    self.X_w2v = X_w2v
    self.X_tfidf = X_tfidf
    self.y = y
    
  def __len__(self):                
    return len(self.y)
    
  def __getitem__(self, index):          
    # your code here
    return self.X_w2v[index], self.X_tfidf[index], self.y[index]

dataset = CustomDataset(X_word2vec, X_tf_idf, y)

split = int(len(dataset)*0.8)
lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128, 
                          collate_fn=collate_fn)

test_loader = DataLoader(test_dataset, shuffle=True, batch_size=128, 
                         collate_fn=collate_fn)

# Model Definition

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM(nn.Module):
  def __init__(self, input_dim, embedding_dim, output_dim):   
    super(BiLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size=input_dim, 
                        hidden_size=embedding_dim,
                        num_layers=1,
                        bidirectional=True,
                        batch_first=True)
    
    self.linear = nn.Linear(embedding_dim*2, 
                            output_dim)
  
  def forward(self, X):
    out, (hn, cn) = self.lstm(X)    
    emb = torch.mean(out, dim=1)
    output = torch.sigmoid(self.linear(emb))
    return output

In [101]:
class DiseasePredictionModel(nn.Module):
  def __init__(self, weight=0.4):    
    super(DiseasePredictionModel, self).__init__()
    self.weight = 0.4    
    self.w2v_lstm = BiLSTM(input_dim=100, embedding_dim=50, output_dim=50)
    self.tf_idf_lstm = BiLSTM(input_dim=50, embedding_dim=50, output_dim=50)
  
  def forward(self, X_w2v, X_tidf):
    pred1 = self.w2v_lstm(X_w2v)
    pred2 = self.tf_idf_lstm(X_tidf)
    # compute the weighted average of predictions
    # from the 2 models.
    return self.weight * pred1 + (1-self.weight) * pred2

# Model Training

In [ ]:
model = DiseasePredictionModel()
model.cuda()
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for e in range(100):
  model.train()
  epoc_train_loss = 0
  for x_w2v, x_tidf, y_batch in train_loader:    
    model.zero_grad()
    pred = model(x_w2v, x_tidf)
    l = loss(pred, y_batch)    
    l.backward()
    optimizer.step()    
    epoc_train_loss += l.item()
  epoc_train_loss = epoc_train_loss/len(train_loader)
  print(f"epoc: {e}: Train Loss: {epoc_train_loss}")

epoc: 0: Train Loss: 0.6910153180360794
epoc: 1: Train Loss: 0.6777748576430387
epoc: 2: Train Loss: 0.6649659951758939
epoc: 3: Train Loss: 0.6520988887479139
epoc: 4: Train Loss: 0.6391024452655815
epoc: 5: Train Loss: 0.6256635500941166
epoc: 6: Train Loss: 0.6119832808888236
epoc: 7: Train Loss: 0.5976034557056982
epoc: 8: Train Loss: 0.5828218709590823
epoc: 9: Train Loss: 0.5674722778589226
epoc: 10: Train Loss: 0.5515962092335834
epoc: 11: Train Loss: 0.5354359373450279
epoc: 12: Train Loss: 0.51892232253801
epoc: 13: Train Loss: 0.5021473158064277
epoc: 14: Train Loss: 0.4856962183880251
epoc: 15: Train Loss: 0.46930311741523967
epoc: 16: Train Loss: 0.4538164575432622
epoc: 17: Train Loss: 0.4385235850201097
epoc: 18: Train Loss: 0.42480946739399156
epoc: 19: Train Loss: 0.41155711145595064
epoc: 20: Train Loss: 0.39955345376632934
epoc: 21: Train Loss: 0.3886476997719255
epoc: 22: Train Loss: 0.3786025263022545
epoc: 23: Train Loss: 0.36968477685437645
epoc: 24: Train Loss: 0

In [73]:
pred, y_batch

(tensor([[0.0729, 0.0968, 0.1757,  ..., 0.0542, 0.0742, 0.0504],
         [0.0547, 0.0849, 0.1845,  ..., 0.0453, 0.0660, 0.0379],
         [0.0703, 0.1011, 0.1832,  ..., 0.0584, 0.0742, 0.0514],
         ...,
         [0.0809, 0.1038, 0.1795,  ..., 0.0606, 0.0802, 0.0565],
         [0.0669, 0.0900, 0.1789,  ..., 0.0518, 0.0723, 0.0434],
         [0.0598, 0.0875, 0.1804,  ..., 0.0470, 0.0698, 0.0399]],
        device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'))

In [76]:
y_batch[0]

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')